In [1]:
import networkx as nx

In [2]:
sandy_m = nx.read_gexf("/home/cadel/synteny/traverse/sandy.deduce.gexf")
consensus_m = nx.read_gexf("/home/cadel/synteny/traverse/ste08_consensus.deduce.gexf")

In [3]:
def get_Ts(genome_map):
    sources = [n for n, d in genome_map.in_degree() if d == 0]
    
    Ts = []
    for source in sources:
        T = [source]
        neighbors = list(genome_map.neighbors(source))
        while len(neighbors) > 0:
            T.append(neighbors[0])
            neighbors = list(genome_map.neighbors(neighbors[-1]))
        Ts.append(T)
    return Ts

Ts = get_Ts(sandy_m)


In [4]:
S = consensus_m

T = Ts[0]
print("Ts", len(Ts))
print("T", len(T))
print("S", len(S))

Ts 18
T 42
S 2848


In [5]:
from operator import itemgetter

S = nx.DiGraph()
S.add_edges_from([
    ('1','3'),
    ('1','2'),
    ('3','6'),
    ('6','5'),
    ('5','3'),
    ('3','4'),
    ('4','8'),
    ('2','4')
])

MATCH = 7
MISMATCH = -3
D = -4


def dbg(O):
    table = []
    for x, ys in O.items():
        row = []
        for y in ys:
            row.append(O[x][y])
        table.append(row)
    
    print("SCORES")
    
    print("   " + " ".join([f"{x:>3}" for x in list(O.values())[0].keys()]))
    for row,t in zip(table, O.keys()):
        print(t + " |" + "|".join(f"{x[0]:>3}" for x in row))
    
    print("BACKLINKS")
    print("   " + " ".join([f"{x:>3}" for x in list(O.values())[0].keys()]))
    for row,t in zip(table, O.keys()):
        print(t + " |" + "|".join(f"{x[1]:>3}" if x[1] is not None else " - " for x in row))

def solve(S, T):
    # Each cell is SCORE, BACKLINK
    O = {t: {s: [0, None] for s in S} for t in T}

    if T[0] in S:
        for s in S:
            O[T[0]][s] = [MATCH, None] if T[0] == s else [I, T[0]]
            
    else:
        for s in S:
            O[T[0]][s] = [I, None]

    dbg(O)

    for i in range(1, len(T)):
        print(f"\nIteration T_{i}")
            
        for s in S:
            #print(f"T[i] = {T[i]}, s = {s}")
            pred = [x[0] for x in S.in_edges(s)]
            if len(pred) == 0:
                best_s_prime = None
                best_score = None
            else:
                best_s_prime = pred[0]
                best_score = O[T[i - 1]][pred[0]][0]
                for s_prime in pred[1:]:
                    if O[T[i - 1]][s_prime][0] > best_score:
                        best_s_prime = s_prime
                        best_score = O[T[i - 1]][s_prime][0]

            pred_t = T[:i]
            best_t_prime = pred_t[0]
            best_t_prime_score = O[pred_t[0]][s][0]
            for t_prime in pred_t[1:]:
                if O[t_prime][s][0] > best_t_prime_score:
                    best_t_prime = t_prime
                    best_t_prime_score = O[t_prime][s][0]

            match_score = MATCH + (best_score or 0) if T[i] == s else MISMATCH + (best_score or 0)
            deletion_score = D + best_t_prime_score
            insertion_score = I + (best_score or 0) 
            
            path_score = max(match_score, deletion_score, insertion_score)
            if path_score == match_score:
                O[T[i]][s] = [match_score, best_s_prime]
            elif path_score == insertion_score:
                O[T[i]][s] = [insertion_score, best_s_prime]
            else:
                O[T[i]][s] = [deletion_score, O[best_t_prime][s]]

        dbg(O)

def solve2(S, T):
    # Each cell is SCORE, BACKLINK
    O = {t: {s: [0, None] for s in S} for t in T}
    
    for i in range(0, len(T)):
        #print(f"\nIteration T_{i}")
            
        for s in S:
            #print(f"T[i] = {T[i]}, s = {s}")
            pred = [x[0] for x in S.in_edges(s)]
            if len(pred) == 0:
                best_s_prime = None
                best_score = None
            else:
                best_s_prime = pred[0]
                best_score = O[T[i - 1]][pred[0]][0]
                for s_prime in pred[1:]:
                    if O[T[i - 1]][s_prime][0] > best_score:
                        best_s_prime = s_prime
                        best_score = O[T[i - 1]][s_prime][0]

            match_score = MATCH + (best_score or 0) if T[i] == s else MISMATCH + (best_score or 0)
            
            if i != 0:
                deletion_score = D + O[T[i-1]][s][0]
                path_score = max(match_score, deletion_score)
                if path_score == match_score:
                    O[T[i]][s] = [match_score, best_s_prime]
                else:
                    O[T[i]][s] = [deletion_score, s]
            else:
                # Has to be the match score
                O[T[i]][s] = [match_score, best_s_prime]

        #dbg(O)
    return O

def backtrack(O):
    #dbg(O)
    xs = list(O.keys())
    ys = list(list(O.values())[0].keys())
    
    best_score = O[xs[0]][ys[0]][0]
    best_end = (0, ys[0])
    
    for i in range(0, len(xs)):
        for y in ys:
            if O[xs[i]][y][0] > best_score or O[xs[i]][y][0] == best_score and i > best_end[0]:
                best_score = O[xs[i]][y][0]
                best_end = (i, y)
    
    backtrace = [best_end]
    
    while O[xs[best_end[0]]][best_end[1]][1] is not None and best_end[0] >= 0:
        best_end = (best_end[0] - 1, O[xs[best_end[0]]][best_end[1]][1])
        backtrace.append(best_end)
        
    path = [x[1] for x in reversed(backtrace)]
    
    return best_score, path

# Match all
print("PATH", ['1','3','4','8'])
backtrack(solve2(S,['1','3','4','8']))

# Skip in S = deletion
print("PATH", ['1','4','8'])
backtrack(solve2(S,['1','4','8']))

print("PATH", ['1','4'])
backtrack(solve2(S,['1','4']))
dbg(solve2(S,['1','4']))

# # Skip in T = insertion
print("PATH", ['1','3', '5', '4','8'])
backtrack(solve2(S,['1','3', '5', '4','8']))
print("PATH", ['1','3', '7', '4','8'])
backtrack(solve2(S,['1','3', '7', '4','8']))



PATH ['1', '3', '4', '8']
PATH ['1', '4', '8']
PATH ['1', '4']
SCORES
     1   3   2   6   5   4   8
1 |  7| -3| -3| -3| -3| -3| -3
4 |  3|  4|  4| -6| -6|  4| -6
BACKLINKS
     1   3   2   6   5   4   8
1 | - |  1|  1|  3|  6|  3|  4
4 |  1|  1|  1|  3|  6|  3|  4
PATH ['1', '3', '5', '4', '8']
PATH ['1', '3', '7', '4', '8']


(24, ['1', '3', '3', '4', '8'])

In [6]:
import os
import pandas as pd

consensus_m = nx.read_gexf("/home/cadel/synteny/traverse/ste08_consensus.deduce.gexf")

sandy_maps = [
    "sandy.v0a.0.canu.7e2cdc91b4fdbfc6f030d41258c93f66.bed",
    "sandy.v0a.1.arrow.3a9e7242d79bcee1e28198716e91ca6e.bed",
    "sandy.v0a.2.purged.bfac1ac7dc7f9e554f275c4d5c0340e2.bed",
    "sandy.v0a.3.hic.1903162ece8eae5d9bf95f5ea3af27d4.bed",
    "sandy.v0a.4.arrow2.b5a0d8c8cd4fa28ce2f832f67486a376.bed",
    "sandy.v0a.5.pbjelly.3ea9048295a3bbe12288339b2305e10d.bed",
    "sandy.v0a.6.arrow3.c6dc12af067cd7d74e7963ff3796a427.bed",
    "sandy.v0a.7.pilon.909a98978daf059ea91ef719d5a1fb07.bed",
    "sandy.v0a.8.pafscaff.a53533b87d28438674007544d5506fe2.bed",
    "sandy.v2.0.e8a5a59ffbb048bbcb8cbeef281d626e.bed",
    "sandy.v2.1.4baab19c4fd7e3a89f9fd466c9b40ec2.bed",
    "sandy.v2.2.7feb0d1ddaa74795fcf2d2b1a16a5519.bed"
]

records = []

for m in sandy_maps:
    
    sandy_m = nx.read_gexf(os.path.join("/home/cadel/synteny/sandy/maps", m + ".gexf"))
    Ts = get_Ts(sandy_m)

    print(f"=> {m} ({len(Ts)} paths)")
    for T in Ts:
        best_score, path = backtrack(solve2(consensus_m, T))
        records.append([m, T[0], best_score, len(path)])
        
df = pd.DataFrame.from_records(records, names=["iteration", "start", "score", "path_len"])


=> sandy.v0a.0.canu.7e2cdc91b4fdbfc6f030d41258c93f66.bed (126 paths)
=> sandy.v0a.1.arrow.3a9e7242d79bcee1e28198716e91ca6e.bed (125 paths)
=> sandy.v0a.2.purged.bfac1ac7dc7f9e554f275c4d5c0340e2.bed (126 paths)
=> sandy.v0a.3.hic.1903162ece8eae5d9bf95f5ea3af27d4.bed (40 paths)
=> sandy.v0a.4.arrow2.b5a0d8c8cd4fa28ce2f832f67486a376.bed (40 paths)
=> sandy.v0a.5.pbjelly.3ea9048295a3bbe12288339b2305e10d.bed (39 paths)
=> sandy.v0a.6.arrow3.c6dc12af067cd7d74e7963ff3796a427.bed (40 paths)
=> sandy.v0a.7.pilon.909a98978daf059ea91ef719d5a1fb07.bed (40 paths)
=> sandy.v0a.8.pafscaff.a53533b87d28438674007544d5506fe2.bed (40 paths)
=> sandy.v2.0.e8a5a59ffbb048bbcb8cbeef281d626e.bed (39 paths)
=> sandy.v2.1.4baab19c4fd7e3a89f9fd466c9b40ec2.bed (39 paths)
=> sandy.v2.2.7feb0d1ddaa74795fcf2d2b1a16a5519.bed (40 paths)


TypeError: from_records() got an unexpected keyword argument 'names'

In [20]:
df = pd.DataFrame.from_records(records, columns=["iteration", "start", "score", "path_len"])


In [32]:
import seaborn as sns

g = df.groupby("iteration").agg(
    n_paths = pd.NamedAgg('path_len', 'count'),
    avg_len = pd.NamedAgg('path_len', 'mean'),
    avg_score = pd.NamedAgg('score', 'mean')
).reset_index()

g

,iteration,n_paths,avg_len,avg_score
0,sandy.v0a.0.canu.7e2cdc91b4fdbfc6f030d41258c93...,126,19.833333,118.730159
1,sandy.v0a.1.arrow.3a9e7242d79bcee1e28198716e91...,125,19.944000,119.808000
2,sandy.v0a.2.purged.bfac1ac7dc7f9e554f275c4d5c0...,126,19.912698,120.325397
3,sandy.v0a.3.hic.1903162ece8eae5d9bf95f5ea3af27...,40,63.725000,370.975000
4,sandy.v0a.4.arrow2.b5a0d8c8cd4fa28ce2f832f6748...,40,63.325000,369.350000
5,sandy.v0a.5.pbjelly.3ea9048295a3bbe12288339b23...,39,62.923077,358.153846
6,sandy.v0a.6.arrow3.c6dc12af067cd7d74e7963ff379...,40,63.350000,348.200000
7,sandy.v0a.7.pilon.909a98978daf059ea91ef719d5a1...,40,62.225000,349.150000
8,sandy.v0a.8.pafscaff.a53533b87d28438674007544d...,40,71.975000,484.725000
9,sandy.v2.0.e8a5a59ffbb048bbcb8cbeef281d626e.bed,39,73.948718,497.358974
